In [5]:
import duckdb
import pandas as pd

%reload_ext sql
con = duckdb.connect("../../data_backup/finance_data.db")
%sql con --alias duckdb
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

In [6]:
%%sql df <<

with scores as (
    select 
        *,
        row_number() over (partition by symbol order by computed_at desc) as rn
    from afv_21_scores 
    order by afv21 desc 
),
    
infos as (
    select
        symbol,
        cast(json_value(data, '$.marketCap.0') as bigint) as market_cap,
        json_extract_string(data, '$.country.0') as country,
        json_extract_string(data, '$.industry.0') as industry,
        json_extract_string(data, '$.averageDailyVolume3Month.0') as avg_daily_volume,
        row_number() over (partition by symbol order by ts desc) as rn
    from yahoo_data
    where dataset = 'info'
)

select
    newest.symbol,
    newest.afv21 as afv21_latest,
    previous.afv21 as afv21_previous,
    case 
        when afv21_latest > previous.afv21 then 'increased' 
        when afv21_latest = previous.afv21 then 'no_change'
        else 'decreased' 
    end as afv21_trend,
    newest.*,
    infos.market_cap / 1e9 as market_cap_billion,
    infos.industry,
    infos.country,
    infos.avg_daily_volume
from scores as newest
left join scores as previous
    on newest.symbol = previous.symbol
    and previous.rn = 2
left join infos
    on infos.symbol = newest.symbol
    and infos.rn = 1
where newest.rn = 1
order by afv21_latest desc
limit 500


In [7]:
from datetime import datetime

date_str = datetime.today().strftime("%Y-%m-%d")
filename = f"../../csv_export/afv21_results_top500_{date_str}.csv"
df.to_csv(filename, index=False)

In [8]:
con.close()